In [71]:
import pickle
import imageio
import sys
import pdb
import os
import cv2
import numpy as np
import trimesh

pickle_name = '15a3f5f564c538e2baa1282ba5a20d4c'

# person1: 15a3f5f564c538e2baa1282ba5a20d4c
# person2: ee17fadac2b5dbee58a4583c94cf5c99
# person3: 5ae85a3a2ea692c0cc75820731d6ed1e
OBJNAME = 'person1'


os.makedirs(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/Annotations/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/JPEGImages/Full-Resolution/{OBJNAME}',exist_ok=True)
os.makedirs(f'output/{OBJNAME}/LASR/Meshes/Full-Resolution/{OBJNAME}',exist_ok=True)
# os.makedirs(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/r{OBJNAME}',exist_ok=True)
# os.makedirs(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/r{OBJNAME}',exist_ok=True)

with open(f'./MeshSeqs/{pickle_name}.pkl', 'rb') as file:
    frames_dict = pickle.load(file)


for i in range(len(frames_dict)):
    imageio.imsave(f'output/{OBJNAME}/LASR/JPEGImages/Full-Resolution/{OBJNAME}/{i:05d}.png',frames_dict[i]['rgb_frame'][...,:-1])
    # miss mask
    imageio.imsave(f'output/{OBJNAME}/LASR/Annotations/Full-Resolution/{OBJNAME}/{i:05d}.png',frames_dict[i]['rgb_frame'][...,-1] // 2)


    # convert connectivity to faces
    # faces = []
    # for arr in map(lambda x:np.unique(np.concatenate(np.asarray(x))),frames_dict[i]['mesh_connectivity_graph'][0]):
    #     if len(arr) != 3:
    #         tri = trimesh.geometry.triangulate_quads([arr])
    #         faces.append(tri)
    #         # pdb.set_trace()
    #     else:
    #         faces.append([arr])

    # faces = np.concatenate(faces)

    connectivity_to_vertices = lambda x:np.unique(np.concatenate(np.asarray(x)))
    quads = list(map(connectivity_to_vertices, frames_dict[i]['mesh_connectivity_graph'][0]))
    faces = trimesh.geometry.triangulate_quads(quads)
    
    mesh = trimesh.Trimesh(vertices=frames_dict[i]['mesh_coords_3d'][0,...], faces=faces)
    trimesh.exchange.export.export_mesh(mesh,f'output/{OBJNAME}/LASR/Meshes/Full-Resolution/{OBJNAME}/{i:05d}.obj')

# save gif
imageio.mimsave(f'output/{OBJNAME}/{pickle_name}.gif',[frames_dict[i]['rgb_frame'][...,:-1] for i in range(len(frames_dict))])



fw = np.array([frames_dict[i]['forward_optical_flow'] for i in range(len(frames_dict))])[:-1,...]
bw = np.array([frames_dict[i]['backward_optical_flow'] for i in range(len(frames_dict))])[1:,...]


# write optical flow and occlusion map in LASR format
def write_pfm(path, image, scale=1):
    """Write pfm file.

    Args:
        path (str): pathto file
        image (array): data
        scale (int, optional): Scale. Defaults to 1.
    """

    with open(path, "wb") as file:
        color = None

        if image.dtype.name != "float32":
            raise Exception("Image dtype must be float32.")

        image = np.flipud(image)

        if len(image.shape) == 3 and image.shape[2] == 3:  # color image
            color = True
        elif (
            len(image.shape) == 2 or len(image.shape) == 3 and image.shape[2] == 1
        ):  # greyscale
            color = False
        else:
            raise Exception("Image must have H x W x 3, H x W x 1 or H x W dimensions.")

        file.write("PF\n".encode() if color else "Pf\n".encode())
        file.write("%d %d\n".encode() % (image.shape[1], image.shape[0]))

        endian = image.dtype.byteorder

        if endian == "<" or endian == "=" and sys.byteorder == "little":
            scale = -scale

        file.write("%f\n".encode() % scale)

        image.tofile(file)

if 'no-flo' not in OBJNAME:

    for i in range(len(fw)):
        f = fw[i,...]
        ones = np.ones_like(f[...,:1])  
        f = np.concatenate([f[...,1:], f[...,:1], ones],-1)
        b = np.concatenate([-bw[i,...,1:],-bw[i,...,:1], ones],-1)

        f = np.flip(f,0)
        b = np.flip(b,0)
        
        write_pfm(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/{OBJNAME}/flo-{i:05d}.pfm',f)
        write_pfm(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/{OBJNAME}/flo-{i+1:05d}.pfm',b)
        write_pfm(f'output/{OBJNAME}/LASR/FlowFW/Full-Resolution/{OBJNAME}/occ-{i:05d}.pfm',np.ones_like(f[...,0]))
        write_pfm(f'output/{OBJNAME}/LASR/FlowBW/Full-Resolution/{OBJNAME}/occ-{i+1:05d}.pfm',np.ones_like(b[...,0]))


In [3]:
for k in frames_dict[0]:
    print(k)

rgb_frame
model_names
animation_names
keypoint_names
poses_3d
poses_2d
bounding_boxes_2d
camera_intrinsics
camera_extrinsics
focal_lens_mm
mesh_coords_3d
num_vertices
mesh_connectivity_graph
mesh_normals
backward_optical_flow
forward_optical_flow


In [62]:
faces = []
for arr in map(lambda x:np.unique(np.concatenate(np.asarray(x))),frames_dict[0]['mesh_connectivity_graph'][0]):
    if len(arr) != 3:
        tri = trimesh.geometry.triangulate_quads([arr])
        faces.append(tri)
        # pdb.set_trace()
    else:
        faces.append([arr])

faces = np.concatenate(faces)

In [70]:
trimesh.geometry.triangulate_quads(list(map(lambda x:np.unique(np.concatenate(np.asarray(x))),frames_dict[0]['mesh_connectivity_graph'][0])))

(8404, 3)

In [63]:
faces.shape

(7040, 3)

In [12]:
frames_dict[0]['num_vertices']

array([3534])

In [32]:
import pickle
import imageio
import sys
import pdb
import os
import cv2
import numpy as np
import trimesh
import glob

os.makedirs(f'./MeshSeqs/preview/',exist_ok=True)

for filename in list(glob.glob('./MeshSeqs/*.pkl')):
    with open(filename, 'rb') as f:
        frames_dict = pickle.load(f)
    
    pickle_name = filename.split('/')[-1]
    imageio.mimsave(f'./MeshSeqs/preview/{pickle_name}.gif',[frames_dict[i]['rgb_frame'][...,:-1] for i in range(len(frames_dict))])



In [31]:
filename.split('/')

['.', 'MeshSeqs', '10885fd4bb07292e866af001f841aa24.pkl']

In [5]:
mesh

<trimesh.Trimesh(vertices.shape=(3204, 3), faces.shape=(0, 3))>

In [11]:
cv2.imread('output/airplane/LASR/Annotations/Full-Resolution/airplane/00000.png').max()
# cv2.imread(f'output/{OBJNAME}/LASR/JPEGImages/Full-Resolution/{OBJNAME}/{i:05d}.png')


128